In [ ]:
import pandas as pd 
import pickle
import datetime
import re
import networkx as nx
import numpy as np
from tqdm import tqdm

# Atrial fibrillation (AF) patients on rivaroxaban

In [ ]:
with open('/projects/ROCKET AF/1A/rivaroxaban dataextraction/data/first_mention_rivaroxaban.pickle', 'rb') as f:
    rivaroxaban = pickle.load(f)

In [ ]:
len(rivaroxaban)

In [ ]:
df = pd.DataFrame(list(rivaroxaban.items()), columns=['client_idcode', 'first_mention_rivaroxaban']) # This code converts a dictionary (rivaroxaban) into a Pandas DataFrame with two columns named 'client_idcode' and 'first_mention_rivaroxaban'
df.shape

In [ ]:
ids = set(df['client_idcode'])
len(ids)

#  eGFR

In [ ]:
df = pd.read_csv('/projects/ROCKET AF/data/GFR.csv')

In [ ]:
df.head()

In [ ]:
df['basicobs_itemname_analysed'].unique()

In [ ]:
df.loc[(df['basicobs_itemname_analysed'] == 'Estimated GFR - POCT1')]

In [ ]:
df = df.loc[(df['basicobs_itemname_analysed'] == 'Estimated GFR')]

In [ ]:
df.shape

In [ ]:
df['basicobs_itemname_analysed'].unique()

In [ ]:
df = df[['client_idcode', 'basicobs_itemname_analysed', 'basicobs_value_numeric', 'basicobs_unitofmeasure', 'basicobs_value_analysed', 'basicobs_createdwhen']]

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['basicobs_unitofmeasure'].unique()

In [ ]:
df = df.loc[df['basicobs_unitofmeasure'].notna()]
df.shape

In [ ]:
df['first_rivaroxaban_prescription'] = df['client_idcode'].map(rivaroxaban)

In [ ]:
df.head()

In [ ]:
df['basicobs_createdwhen'] = pd.to_datetime(df['basicobs_createdwhen'], utc=True).dt.date 

In [ ]:
df.head()

In [ ]:
df['difference'] = pd.to_datetime(df['first_rivaroxaban_prescription']) - pd.to_datetime(df['basicobs_createdwhen'])

In [ ]:
df.loc[df['difference'].notna()]

In [ ]:
min_time = datetime.timedelta(days=1) 
max_time = datetime.timedelta(days=183)
t1 = df['difference'] >= min_time 
t2 = df['difference'] <= max_time
in_window = t1 & t2 
df['in_window'] = in_window

In [ ]:
df = df.loc[df['in_window'] == True]

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# Group by 'client_idcode' and keep only the rows with the smallest 'difference'

min_values = df.groupby('client_idcode')['difference'].transform('min')

# Filter the DataFrame to keep only rows with the minimum 'Value'
df = df[df['difference'] == min_values]

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['client_idcode'].nunique()

In [ ]:
df = df.sort_values(by='client_idcode')
df.head(10)

In [ ]:
df.info()

In [ ]:
# Check whether the 'basicobs_unitofmeasure' column has the same value for all rows with the same client_idcode

result = df.groupby('client_idcode')['basicobs_unitofmeasure'].nunique() == 1
ids_with_same_unitofmeasure = result[result].index.tolist()

In [ ]:
len(ids_with_same_unitofmeasure)

In [ ]:
# Group by 'client_idcode' and calculate the average of 'eg count'

df = df.groupby('client_idcode').agg({'basicobs_value_numeric': 'mean', 'basicobs_unitofmeasure': 'first'}).reset_index()
    

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['client_idcode'].nunique()

In [ ]:
df['basicobs_value_numeric'].max()

In [ ]:
df['basicobs_value_numeric'].min()

In [ ]:
df.info()

In [ ]:
df.loc[df['basicobs_value_numeric'] < 60].shape

In [ ]:
eGFR = dict(zip(df['client_idcode'], df['basicobs_value_numeric']))
eGFR

In [ ]:
with open('/projects/ROCKET AF/1A/rivaroxaban dataextraction/data/eGFR.pickle', 'wb') as f:
    pickle.dump(eGFR,f)